In [8]:
# Start writing code here...
!pip install -r ../requirements.txt

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from igraph import *
import disjoint_set

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [9]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
os.listdir("../data")

['sources.csv.gz',
 'input_wagon.xlsx',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'wagon_mode_compat.csv.gz',
 'description.txt',
 'output_wagon.xlsx',
 'metrics.csv.gz',
 'standplaces.csv.gz']

In [12]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz")
sources = pd.read_csv("../data/sources.csv.gz")
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz")

In [13]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.common import *



In [14]:
# get_roads_graph


In [15]:
from igraph import *
from utils.abstractions import get_roads_graph


graph, name2vertex = get_roads_graph(metrics)


0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000


In [16]:
total_sum = 0

stale_price = {}
stale_move = {}
for index, row in sources.iterrows():
    stale_price[row['Station']] = int(row['Stand']) 


for index, row in metrics.iterrows():
    stale_move[str(row['From']) + "_" + str(row['To'])] = int(row['PriceUnit']) 
    stale_move[str(row['To']) + "_" + str(row['From'])] = int(row['PriceUnit']) 


In [17]:
from utils.common import *

order_manager = OrderManager()
for index, row in orders7.iterrows():
    order = Order2(row['Start'], row['Finish'], row['StartDate'], int(row['MaxUnit']), row['NeedWagonModel'], int(row['Tariff'].split(":")[0]), row['Dur'], row['ShortagePenalty'], row['OrderNum'])
    order_manager.add_order(order)
    total_sum += order.total_profit()
print(total_sum)

wagon_manager = WagonManager()
for index, row in sources.iterrows():
    for units in range(row['Units']):
        occupied = True
        if int(row['Date']) == 0:
            occupied = False
        wagon = Wagon(row['WagonModel'], str(row['Station']), occupied, row['Date'])
        wagon_manager.add_wagon(wagon)

print(wagon_manager.wagon_number())


comp_wagon = set()
for index, row in wagon_mode_compat.iterrows():
    comp_wagon.add(str(row['NeedWagonModel']) + "_" + str(row['CompatibleWagonModel']))



strategy =  Strategy(order_manager, wagon_manager, graph, comp_wagon, name2vertex, stale_price, stale_move)
print(strategy.compability_model)
for day in range(28):
    strategy.run_day(day)
    print(day)
    #break
print("Bad orders - " +  str(strategy.bad_orders))
print("Total orders - " +  str(strategy.total_orders))

print("Profit - " + str(strategy.total_profit))
print("Stale stop - " + str(strategy.stale_stop))
print("Stale move - " + str(strategy.stale_move))
print("Profit per cat - " + str(strategy.profit_per_cat))


434839750
15438
{'2444_2149', '2068_2143', '2029_2157', '2307_2068', '2360_2067', '2172_2139', '2351_2160', '2134_2066', '2379_2161', '2073_2049', '2254_2136', '2019_2068', '2201_2137', '2008_2184', '2450_2166', '2430_2158', '2459_2167', '2345_2061', '2295_2066', '2185_2152', '2130_2160', '2197_2166', '2471_2160', '2109_2155', '2060_2049', '2400_2071', '2447_2149', '2260_2135', '2218_2191', '2078_2070', '2150_2164', '2318_2166', '2355_2071', '2004_2091', '2052_2088', '2277_2064', '2304_2142', '2113_2154', '2262_2037', '2186_2166', '2259_2144', '2118_2155', '2108_2167', '2023_2049', '2276_2059', '2449_2146', '2266_2162', '2185_2166', '2209_2158', '2016_2139', '2022_2071', '2306_2146', '2158_2164', '2428_2136', '2010_2038', '2029_2053', '2211_2145', '2036_2071', '2055_2144', '2166_2167', '2137_2067', '2312_2143', '2354_2061', '2074_2068', '2436_2157', '2010_2146', '2045_2161', '2095_2064', '2328_2149', '2256_2149', '2028_2153', '2341_2062', '2316_2046', '2281_2157', '2258_2062', '2195_21

In [18]:
strategy.stale_moves_num

8954

In [19]:
with open("orders.csv", 'w') as orders:
    orders.write("Код заказа,Станция старта,Станция финиша,Расчетная дата старта,Плановая дата старта,Время в пути,Мин допустимый объем,Запрошенный объем,Тарифы,Запрошенный тип вагона,Приоритет,Примененный тариф\n")
    for order in strategy.ord_log:
        orders.write(order + "\n")
        


In [20]:
with open("empty_moves.csv", 'w') as orders:
    orders.write("Отправление,Прибытие,Тип простоя,Модель вагона,День отправления (от начала расчета),День прибытия (от начала расчета),Количество,Цена порожнего пробега (за вагон),Код следующего заказa,Расчитанный тариф следующего заказа\n")
    for order in strategy.stale_move_log:
        orders.write(order + "\n")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>